### Import required libraries

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

### Build scraper

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
page_text = page.text
soup = bs(page_text, 'html.parser')

### Create DataFrame

In [12]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head() 

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Clean and Wrangle dataframe
1. Drop all postal codes without a Borough. Check to see if any postal code has multiple rows
2. Combine neighborhoods that share the same postal code
3. Any postal code that has a borough, but not a neighborhood is assigned that borough as its neighborhood


In [36]:
df1 = df[df['Borough'] != 'Not assigned']
df1['Postal Code'].value_counts(sort = True)

df2 = df1.groupby(['Postal Code', 'Borough']).agg({
    'Neighbourhood': lambda x: ', '.join(x)
})
df2 = df2.reset_index()

df3 = df2
df3['Neighbourhood'] = np.where(df3['Neighbourhood'] == 'Not assigned', df3['Borough'], df3['Neighbourhood'])

df3.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Final DataFrame and shape

In [37]:
final_df = df3
final_df.shape

(103, 3)